In [ ]:
import numpy as np
np.random.seed(0)

In [ ]:
class SSQ:
  def __init__(self):
    n_list = np.random.exponential(.5, 100)   #exponential distribution mean 1/3
    self.interarrivals = n_list.tolist()
    s_list= np.random.exponential(1.3, 100)   #exponential distribution mean 1/4
    self.service_times = s_list.tolist()
    #print(self.interarrivals)
    #print(self.service_times)

   # self.interarrivals = [0.397937254081555, 0.6279653814829189, 0.4616115729020344, 0.39360057615863536, 0.2755242455477496, 0.5190796474718047, 0.2877595995843199, 1.1117622065163284, 1.657456091026907, 0.24180105045112665, 0.7844480699845842, 0.3763370547682942, 0.41971642336941145, 1.2991270769627203, 0.03684267745642394, 0.04558051455950024, 0.010212793342112475, 0.8937438398074441, 0.7528921159326077, 1.0201571404727754]
  #  self.service_times = [4.998788389616904, 2.08681142663519, 0.8046082961968591, 1.9714967699441972, 0.16363673537970563, 1.3278614523096948, 0.20114858754593876, 3.762746584598984, 0.9591754618043763, 0.6962354337909832, 0.3994644590175899, 1.9347312903447555, 0.7918071411940105, 1.0924351189278956, 0.02465914316109906, 1.2497952079533172, 1.2310956811969116, 1.2474123645480066, 3.7412896023633304, 1.4886806462454891]
    #select Algorithms
    print("press '1' for FIFO")
    print("press '2' for LIFO")
    print("press '3' for SJF")
    print("press '4' for SIRO")
    
    print("Enter a number for selecting an algorithm : ")
    self.Algorithm = int(input())



    self.clock= 0.0
    self.clock_list = [0]
    
    self.next_arrival=self.interarrivals.pop(0)
    self.next_departure= float('inf')
    self.next_departure_1=float('inf')
    
    self.num_in_queue= 0
    self.times_of_arrivalqueue= []         #store times of arrivals who are waiting in the queue
    self.service_times_in_queue= []   #store service times of waiting customers in the queue
    
    self.total_delay=0.0
    self.num_of_delays= 0.0
    self.area_under_q= 0.0
    self.area_under_b1 = 0.0
    self.area_under_b2 = 0.0
    
    #server...1
    self.server_status_1= 0        #0 for IDLE , 1 for BUSY
    self.server_1 = [0]
    
    #server...2
    self.server_status_2=0
    self.server_2=[0]


    self.last_event_time=0.0     #we will need to store last event clock time 

    self.Q_t = [0]                    # for Q_t vs t graph

  def start(self):
    n = 59
    while self.num_of_delays<=n:
      self.timing() 
    
    # Average Delay Calculation
    Average_delay = self.total_delay/(self.num_of_delays)
    print("Average_delay = ",Average_delay)

    # Expected number of customers in the queue Calculation
    expected_customers = self.area_under_q/self.last_event_time
    print("Expected number of customers in the queue = ",expected_customers)

    # Expected utilization of the server1 Calculation
    expected_utilization1 = self.area_under_b1/self.last_event_time
    print("expected utilization of the server1 = ",expected_utilization1)

    # Expected utilization of the server2 Calculation
    expected_utilization2 = self.area_under_b2/self.last_event_time
    print("expected utilization of the server2 = ",expected_utilization2)
    
     
   
    
    

  def timing(self):
    self.clock= min(self.next_arrival,self.next_departure,self.next_departure_1)   #First set clock to minimum time of next event
    self.clock_list.append(self.clock)
    self.update_register()

      
    if self.next_arrival<= self.next_departure and self.next_arrival<=self.next_departure_1:
      self.arrival()
      #print("Arrival at Clock:" +str(self.clock))
    
    elif self.next_departure<=self.next_arrival and self.next_departure<=self.next_departure_1:
      self.departure()
      #print("Departure at "+str(self.clock))

    else:
      self.departure1()
      #print("Departure at "+str(self.clock))



    print("Arrival at Clock     : "+ str(self.clock))
    print("Next Arrival time    : "+ str(self.next_arrival))
    print("Next Departure time  : "+ str(self.next_departure))
    print("Next Departure1 time : "+ str(self.next_departure_1))
    print("Server1 Status     : "+ str(self.server_status_1))
    print("Server2 Status     : "+ str(self.server_status_2))
    print("Times of arrivals in Queue: "+ str(self.times_of_arrivalqueue))
    print("Service times in Queue: "+str(self.service_times_in_queue))
    print("Total Delay:" +str(self.total_delay))
    print("Number of customers delayed  : "+ str(self.num_of_delays))
    print(" ")

  def arrival(self):
    #Schedule next arrival , new_arrival = previous_arrival + inter_arrival time of next customer
    self.next_arrival+= self.interarrivals.pop(0)  
    
    if self.server_status_1==0:                                         #server is idle
      self.server_status_1= 1                                           #make server BUSY
      delay=0.0                                                         #so delay is zero
      self.total_delay += delay  
      self.num_of_delays +=1                                            #increase the number of customers delayed
      #schedule next departure, pop the first element of service_times list to get service time of this customer
      self.next_departure = self.clock+ self.service_times.pop(0)


    elif self.server_status_2==0:
      self.server_status_2 = 1
      delay=0.0
      self.total_delay += delay
      self.num_of_delays += 1 
      self.next_departure_1 = self.clock + self.service_times.pop(0)                                      
  
    else:     #Server is BUSY
      #increase queue length, this customer will have to wait in the queue
      self.num_in_queue+=1


      #store the arrival time and service time of this customer in seperate lists
      self.times_of_arrivalqueue.append(self.clock) 
      self.service_times_in_queue.append(self.service_times.pop(0)) 
    self.server_1.append(1)
    q_new = len(self.times_of_arrivalqueue)
    #print("Q_t = ",q_new)
    self.Q_t.append(q_new)

  def departure(self):
    #check number of customers in the queue
    if self.num_in_queue==0:  #if no customer in the queue
      #make server IDLE 
      self.server_status_1= 0
      #schedule next departure= infinity
      self.next_departure= float('infinity')
      self.server_1.append(0)
    
    else: 
      #if queue not empty, pop one customer, decrease queue length
      self.num_in_queue-=1
      self.num_of_delays+=1
      self.server_1.append(1)
      #AS FIFO, pop first arrival and service time from the queue. IF LIFO we have to pop last arrival and service time
      #For SJF, finf the index of minimum service time from  service_times_in_queue list.
      #Then pop the arrival of that index from times_of_arrivalqueue for delay count and others.  
      
      #For FIFO
      if self.Algorithm == 1:
        arrival= self.times_of_arrivalqueue.pop(0)
        delay= self.clock- arrival
        self.total_delay+=delay
        self.next_departure= self.clock+ self.service_times_in_queue.pop(0)
      
      #For LIFO
      elif self.Algorithm == 2 :
        index = len(self.times_of_arrivalqueue)-1 #finding the last index numner in queue
        arrival= self.times_of_arrivalqueue.pop(index)
        delay= self.clock- arrival
        self.total_delay+=delay
        self.next_departure= self.clock+ self.service_times_in_queue.pop(index)
      
      #For SJF
      elif self.Algorithm == 3 :
        minimum = min(self.service_times_in_queue)
        index = self.service_times_in_queue.index(minimum)
        arrival= self.times_of_arrivalqueue.pop(index)
        delay= self.clock- arrival
        self.total_delay+=delay
        self.next_departure= self.clock+ self.service_times_in_queue.pop(index)

      #For SIRO
      elif self.Algorithm == 4:
        l = len(self.times_of_arrivalqueue)  #finding the lenth of queue
        index = np.random.randint(l)         #random number generation
        arrival= self.times_of_arrivalqueue.pop(index)
        delay= self.clock- arrival
        self.total_delay+=delay
        self.next_departure= self.clock+ self.service_times_in_queue.pop(index)
      


  def departure1(self):
    #check number of customers in the queue
    if self.num_in_queue==0:  #if no customer in the queue
      #make server IDLE 
      self.server_status_2= 0
      #schedule next departure= infinity
      self.next_departure_1= float('infinity')
      self.server_2.append(0)
    
    else: 
      #if queue not empty, pop one customer, decrease queue length
      self.num_in_queue-=1
      self.num_of_delays+=1
      self.server_2.append(1)
      #AS FIFO, pop first arrival and service time from the queue. IF LIFO we have to pop last arrival and service time
      #For SJF, finf the index of minimum service time from  service_times_in_queue list.
      #Then pop the arrival of that index from times_of_arrivalqueue for delay count and others.  
      
      #For FIFO
      if self.Algorithm == 1:
        arrival= self.times_of_arrivalqueue.pop(0)
        delay= self.clock- arrival
        self.total_delay+=delay
        self.next_departure_1= self.clock+ self.service_times_in_queue.pop(0)
      
      #For LIFO
      elif self.Algorithm == 2 :
        index = len(self.times_of_arrivalqueue)-1 #finding the last index numner in queue
        arrival= self.times_of_arrivalqueue.pop(index)
        delay= self.clock- arrival
        self.total_delay+=delay
        self.next_departure_1= self.clock+ self.service_times_in_queue.pop(index)
      
      #For SJF
      elif self.Algorithm == 3 :
        minimum = min(self.service_times_in_queue)
        index = self.service_times_in_queue.index(minimum)
        arrival= self.times_of_arrivalqueue.pop(index)
        delay= self.clock- arrival
        self.total_delay+=delay
        self.next_departure_1= self.clock+ self.service_times_in_queue.pop(index)

      #For SIRO
      elif self.Algorithm == 4:
        l = len(self.times_of_arrivalqueue)  #finding the lenth of queue
        index = np.random.randint(l)         #random number generation
        arrival= self.times_of_arrivalqueue.pop(index)
        delay= self.clock- arrival
        self.total_delay+=delay
        self.next_departure_1= self.clock+ self.service_times_in_queue.pop(index)
      






    q_new = len(self.times_of_arrivalqueue)
    #print("Q_t = ",q_new)
    self.Q_t.append(q_new)

  def update_register(self):
    time_differnce= self.clock - self.last_event_time
    self.area_under_q+=self.num_in_queue*time_differnce
    self.area_under_b1+=self.server_status_1 * time_differnce
    self.area_under_b2+=self.server_status_2 * time_differnce
    self.last_event_time=self.clock

In [ ]:
s = SSQ()
s.start()